In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

stock_data = pd.read_csv('fulldata.csv')
stock_data["FutureOne"]= stock_data.Close.shift(-1)
stock_data.loc[stock_data.Close >= stock_data.FutureOne, "TrendOne"]= 0
stock_data.loc[stock_data.Close < stock_data.FutureOne, "TrendOne"]= 1
stock_data['TrendOne']=stock_data['TrendOne'].fillna(0).astype(int)

stock_data

,DateTime,Volume,Transaction,Open,High,Low,Close,Dir,Change,ForeignBuy,ForeignSell,ForeignNet,TrustBuy,TrustSell,TrustNet,DealerNet,DealerBuy,DealerSell,InvestorsNet,FutureOne,TrendOne
0,2013-01-02,40647,14353,97.6,99.9,97.1,99.6,+,2.6,22484,8625,13859,1746,147,1599,-350,1300,1650,15108,101.0,1
1,2013-01-03,44335,12616,100.5,102.0,100.0,101.0,+,1.4,32303,17347,14955,926,516,410,-1067,1512,2579,14298,101.5,1
2,2013-01-04,39502,10276,100.5,101.5,100.0,101.5,+,0.5,33581,24337,9244,234,190,44,-246,910,1156,9042,100.5,0
3,2013-01-07,42328,9042,101.0,101.0,99.1,100.5,-,1.0,26613,26952,-339,203,925,-722,-450,871,1321,-1511,99.7,0
4,2013-01-08,31218,9463,99.6,100.0,98.9,99.7,-,0.8,16628,15087,1540,209,635,-426,-80,749,829,1034,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,2023-03-30,20380,18171,536.0,537.0,531.0,535.0,+,5.0,16398,10674,5724,32,131,-99,146,520,374,5770,533.0,0
2505,2023-03-31,24324,15962,538.0,538.0,532.0,533.0,-,2.0,20121,16292,3829,60,144,-84,757,869,112,4502,530.0,0
2506,2023-04-06,26706,19144,530.0,531.0,526.0,530.0,-,3.0,22033,19679,2353,60,89,-29,-221,193,414,2103,531.0,1
2507,2023-04-07,11114,10285,535.0,535.0,529.0,531.0,+,1.0,8456,6175,2281,147,103,44,-94,56,150,2231,529.0,0


In [2]:
# define the target and features
DateTime=['DateTime']
target = ['TrendOne']
features = ['Transaction','Open', 'High', 'Low','Close','TrustBuy','TrustNet']
df = stock_data[DateTime+features + target]
df

,DateTime,Transaction,Open,High,Low,Close,TrustBuy,TrustNet,TrendOne
0,2013-01-02,14353,97.6,99.9,97.1,99.6,1746,1599,1
1,2013-01-03,12616,100.5,102.0,100.0,101.0,926,410,1
2,2013-01-04,10276,100.5,101.5,100.0,101.5,234,44,0
3,2013-01-07,9042,101.0,101.0,99.1,100.5,203,-722,0
4,2013-01-08,9463,99.6,100.0,98.9,99.7,209,-426,1
...,...,...,...,...,...,...,...,...,...
2504,2023-03-30,18171,536.0,537.0,531.0,535.0,32,-99,0
2505,2023-03-31,15962,538.0,538.0,532.0,533.0,60,-84,0
2506,2023-04-06,19144,530.0,531.0,526.0,530.0,60,-29,1
2507,2023-04-07,10285,535.0,535.0,529.0,531.0,147,44,0


In [3]:
# split the data
split = int(df.shape[0] * 0.8)
df_train = df.iloc[:split, :].copy()
df_test = df.iloc[split:, :].copy()
# scale the data
target_scaler = MinMaxScaler().fit(df_train[target])
df_train[target] = target_scaler.transform(df_train[target])
df_test[target] = target_scaler.transform(df_test[target])

features_scaler = MinMaxScaler().fit(df_train[features])
df_train[features] = features_scaler.transform(df_train[features])
df_test[features] = features_scaler.transform(df_test[features])

df_train

,DateTime,Transaction,Open,High,Low,Close,TrustBuy,TrustNet,TrendOne
0,2013-01-02,0.056705,0.008056,0.008560,0.007406,0.008987,0.353799,0.644491,1.0
1,2013-01-03,0.049285,0.013135,0.012155,0.012520,0.011407,0.187639,0.513095,1.0
2,2013-01-04,0.039290,0.013135,0.011299,0.012520,0.012271,0.047416,0.472649,0.0
3,2013-01-07,0.034018,0.014011,0.010443,0.010933,0.010543,0.041135,0.387999,0.0
4,2013-01-08,0.035817,0.011559,0.008731,0.010580,0.009160,0.042351,0.420709,1.0
...,...,...,...,...,...,...,...,...,...
2002,2021-03-11,0.200207,0.880911,0.885294,0.885382,0.889388,0.043566,0.482705,1.0
2003,2021-03-12,0.149790,0.914186,0.895566,0.910069,0.898030,0.027356,0.478064,0.0
2004,2021-03-15,0.109672,0.912434,0.888718,0.910069,0.892845,0.021884,0.460935,1.0
2005,2021-03-16,0.155890,0.905429,0.887006,0.906542,0.896301,0.031814,0.471654,0.0


In [4]:
# extract the input sequences and output values
sequence_length = 5

X_train, y_train = [], []

for i in range(sequence_length, df_train.shape[0]):
    X_train.append(df_train[features].iloc[i - sequence_length: i])
    y_train.append(df_train[target].iloc[i])

X_train, y_train = np.array(X_train), np.array(y_train)

X_test, y_test = [], []

for i in range(sequence_length, df_test.shape[0]):
    X_test.append(df_test[features].iloc[i - sequence_length: i])
    y_test.append(df_test[target].iloc[i])

X_test, y_test = np.array(X_test), np.array(y_test)

print(X_train.shape)
print(X_test.shape)

(2002, 5, 7)
(497, 5, 7)


In [29]:
# build and train the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=X_train.shape[1:]))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.2))
# model.add(LSTM(units=200, activation="tanh", return_sequences=False))
# model.add(Dropout(0.2))
# model.add(LSTM(units=20, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(units=60, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(units=120, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(Dense(units=50))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 5, 50)             11600     
                                                                 
 dropout_9 (Dropout)         (None, 5, 50)             0         
                                                                 
 lstm_10 (LSTM)              (None, 100)               60400     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 73,021
Trainable params: 73,021
Non-trai

In [30]:
model.compile(optimizer='adam', loss='mean_squared_error')

train_history=model.fit(X_train, y_train, epochs=50, batch_size=64)

model.evaluate(X_test, y_test)

#save the model
# model.save('LSTM_eps50_bsize64.h5')

# # generate the test set predictions
# y_pred = model.predict(X_test)
# y_pred = target_scaler.inverse_transform(y_pred)

Epoch 1/50
32/32 [==============================] - 3s 10ms/step - loss: 0.3220
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2528
Epoch 3/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2512
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2528
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2528
Epoch 6/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2540
Epoch 7/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2519
Epoch 8/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2536
Epoch 9/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2509
Epoch 10/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2522
Epoch 11/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2504
Epoch 12/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2496
Epoch 13/50
32/32 [==================

0.25150710344314575

In [31]:
y_pred = model.predict(X_test)
y_pred = target_scaler.inverse_transform(y_pred)
df['Predicted'] = np.nan
df['Predicted'].iloc[- y_pred.shape[0]:] = y_pred.flatten().round(0)
df

16/16 [==============================] - 0s 2ms/step


,DateTime,Transaction,Open,High,Low,Close,TrustBuy,TrustNet,TrendOne,Predicted
0,2013-01-02,14353,97.6,99.9,97.1,99.6,1746,1599,1,NaN
1,2013-01-03,12616,100.5,102.0,100.0,101.0,926,410,1,NaN
2,2013-01-04,10276,100.5,101.5,100.0,101.5,234,44,0,NaN
3,2013-01-07,9042,101.0,101.0,99.1,100.5,203,-722,0,NaN
4,2013-01-08,9463,99.6,100.0,98.9,99.7,209,-426,1,NaN
5,2013-01-09,7616,100.5,100.5,99.2,100.0,130,-113,1,NaN
6,2013-01-10,8304,99.9,101.0,99.5,101.0,151,-12,0,NaN
7,2013-01-11,7982,101.5,101.5,100.5,101.0,102,-250,1,NaN
8,2013-01-14,6414,101.0,102.0,100.0,102.0,120,120,0,NaN
9,2013-01-15,10021,102.0,102.0,99.8,100.5,300,-199,0,NaN


In [32]:
pd.set_option('display.max_rows', None)
dfpred=df[['DateTime','Open','TrendOne', 'Predicted']].dropna()
dfpred['Predicted']=dfpred['Predicted'].astype(int)
expectedans = dfpred.TrendOne.values
probability = dfpred.Predicted.values
dfpred

,DateTime,Open,TrendOne,Predicted
2012,2021-03-25,572.0,1,0
2013,2021-03-26,582.0,1,0
2014,2021-03-29,599.0,0,0
2015,2021-03-30,600.0,0,0
2016,2021-03-31,596.0,1,0
2017,2021-04-01,598.0,1,0
2018,2021-04-06,615.0,0,1
2019,2021-04-07,614.0,1,1
2020,2021-04-08,606.0,0,1
2021,2021-04-09,618.0,0,1


In [33]:
from sklearn import ensemble, metrics
from sklearn.metrics import accuracy_score
import math

print(metrics.classification_report(expectedans,probability))
print(metrics.confusion_matrix(expectedans, probability))
accuracy = accuracy_score(expectedans, probability)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('precision:',metrics.precision_score(expectedans, probability,average='macro')) 
print('recall:',metrics.recall_score(expectedans, probability,average='macro'))
print('F1-score:',metrics.f1_score(expectedans, probability,average='macro'))

              precision    recall  f1-score   support

           0       0.53      0.18      0.27       276
           1       0.44      0.80      0.57       221

    accuracy                           0.46       497
   macro avg       0.49      0.49      0.42       497
weighted avg       0.49      0.46      0.40       497

[[ 51 225]
 [ 45 176]]
Accuracy: 45.67%
precision: 0.4850763715710723
recall: 0.49058134959669486
F1-score: 0.4200549735504615
